<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>COMP 521 Files and Databases Fall 2020</h1>
<h1 style="font-size: 200%;">Problem Set 4</h1>
</div>

---
By default, SQlite only builds ISAM-tree index. In the following questions,
you will need to write Python3 programs to simulate a hash index on a *(year, fips)* search key for the *Demographics*
table in the <a href="http://csbio.unc.edu/mcmillan/Media/NCCOVID19.db" download>NCCOVID19.db</a> database. 
A function to get the hash-bucket pageid for records in *Demographics*
table is provided below:

<pre>
    def getPageId(year, fips):
        return int((fips + 7*year) % 512)
</pre>

Suppose each *(year, fips, pageId)* tuple in the hash index uses 10 bytes and a single page stores at most 400
*(year, fips, pageId)* pairs.

In [ ]:
import iSQL
query = iSQL.parser("NCCOVID19.db")

---
**Problem 1:** How many buckets will overflow? What is the average number of *(year, fips, pageID)* tuples in a bucket? Suppose a page can hold at most 128 *Demographics* records, on average how many pages you need to fetch to select all *Demographics* of a given county in a given year (you should assume that the county and year are valid entries for the given database)?

In [1]:
import sqlite3

def getPageId(year, fips):
    return int((fips + 7*year) % 512)

db = sqlite3.connect("NCCOVID19.db")
db.row_factory = sqlite3.Row
db.text_factory = str
cursor = db.cursor()
bucket = [0 for i in range(512)]

for row in cursor.execute("SELECT * FROM Demographics"):
    pageId = getPageId(row['year'], row['fips'])
    bucket[pageId] += 1
print(bucket)
print()

overflow = sum([count > 400 for count in bucket])
print('Overflow Buckets:', overflow)

average = sum(bucket)/len(bucket)
print('Average number of search-key-page-id records per bucket:', average)

pageAccess = sum([(count+399)//400 for count in bucket if count > 0])/sum([count > 0 for count in bucket])
print('Average number of Demographics page accesses per year-fips query:', pageAccess)

[1406, 1140, 1596, 1501, 1501, 1349, 1178, 1425, 988, 1653, 1311, 1539, 1197, 1178, 1254, 988, 1463, 1311, 1387, 1235, 1083, 1254, 912, 1482, 1216, 1387, 1140, 1121, 1064, 912, 1330, 1216, 1216, 1140, 988, 1064, 722, 1330, 1140, 1235, 950, 1007, 1064, 722, 1140, 1159, 1064, 950, 836, 1045, 646, 1140, 969, 1064, 760, 855, 874, 646, 950, 969, 931, 760, 779, 874, 570, 969, 779, 931, 722, 779, 741, 570, 779, 779, 741, 741, 589, 741, 494, 798, 589, 760, 551, 589, 665, 494, 665, 589, 608, 551, 532, 665, 380, 684, 399, 608, 456, 532, 475, 380, 494, 399, 418, 456, 342, 475, 190, 494, 266, 418, 266, 361, 323, 190, 342, 266, 304, 266, 190, 323, 76, 342, 152, 304, 190, 190, 133, 76, 190, 152, 114, 190, 38, 133, 0, 190, 0, 114, 0, 38, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

---
**Problem 2:** Assuming that the *Demographics* table occupies 2240 pages. How many passes are required to sort the table by (*year*, *fips*) if 20 buffer-pages are allocated for the sort? What would be the minimum number of buffer pages required to perform the same sort in 2-passes?

In [2]:
import math
print('Number of passes using 20 buffer-pages:', 1 + (math.ceil(math.log(math.ceil(2240 / 20), 20 - 1))))

Number of passes using 20 buffer-pages: 3


---
In the following problems consider evaluation of the following query:

<code>
    SELECT D.year, C.name, SUM(count) as population
    FROM County C, Demographics D
    WHERE C.fips=D.fips
    GROUP BY D.year, D.fips
</code>

Assume the *County* table occupies 3 pages and has 100 records.

---
**Problem 3:** Assume that an index-nested loop is used to evaluate the query as described in the following pseudo code:

<code>
1)    for each page of County:
2)        for each county record in page:
3)            for each year in demographics records:
4)                population = 0
4)                Use year-fips hash index from Problem 1 to find the hash-bucket with relevant Demographics records
5)                Load pages in hash-bucket chain in buffer pages
6)                Sort relevant (year,fips,pageID) tuples by pageID
7)                Read each page with distinct pageID
8)                Scan page for matching year and fips values and add count to population
9)                Output (Demographics.year, County.name, population)
</code>

Which numbered lines from the given pseudo code require loading pages from disk? You can assume that demographic records are approximately clustered by year and fips, and that the pageIDs for *a group of year-fips records will span 6 pages* on average. Estimate the number pages read to perform this query. Note this question is only asking for the number of pages *read*, the number of pages *written* is data dependent.

---
**Problem 4:** Assume that the Sort-Merge loop decribed in **Problem 2** using 20 buffer pages is used to evaluate the query as described in the following pseudo code:

<code>
1)    External Sort Demographics Table by (year, fips)
2)    Read County Table as C and sort in memory by fips
3)    previousYear = 0
4)    for D in sorted demographics records:
5)        if (D.year > previousYear):
6)            previousYear = D.year
7)            fips = C.fips
8)            population = 0
10)       while (D.fips > fips):
11)           Output (D.year, C.name, population)
12)           C.next()
12)           fips = C.fips
13)           population = 0
14)       population += D.count
15)   Output (D.year, C.name, population)
</code>

Which numbered lines from the given pseudo code require loading pages from disk? Estimate the number pages read to perform this query. Note this question is only asking for the number of pages *read*, the number of pages *written* is data dependent.

---
**Problem 5:** Consider the following Functional Dependencies (FDs) that govern the Demographics table whose attributes *(fips, year, race, sex, agelo, agehi, count)* map to the following variable names FYRSLHC: 


<div style="margin:20px 250px">
FYRSL &rarr; FYRSLHC<br>H &rarr; L
</div>

Based on the given FDs, is this table in 3NF? Explain why or why not.

---
**Problem 6:** Next, based on the FDs in **Problem 5**, propose a decomposition of Demographics that achieves BCNF. Express this decompostion as two CREATE TABLE commands with primary key declarations.

---
**Problem 7:** Given your decomposition of Demographics from **Problem 6**, how many rows will be in each resulting table, and is a natural join of these two tables lossless?